European Central Bank (ECB) API with Python
=====

## Data on Eurozone countries

-----

*December 14, 2019*<br>
*@bd_econ*

ECB API Documentation is [here](https://sdw-wsrest.ecb.europa.eu/help/)

This example requests the interest rate by country on 10-year bonds over the period since 2012.

In [1]:
import requests
import pandas as pd

## Parameters/Settings

List of data sets is [here](https://sdw-wsrest.ecb.europa.eu/service/dataflow)

In [2]:
# List of countries
c_list = ['DE', 
          'FR', 
          'IT', 
          'ES', 
          'PT', 
          'GR', 
          'NL', 
          'BE', 
          'AT', 
          'IE']

param = [('dataflow', 'IRS'),
         ('freq', 'M'),
         ('countries', '+'.join(c_list)),
         ('series', 'L'),
         ('trans type', 'L40'),
         ('maturity cat', 'CI'),
         ('counterpart', '0000'),
         ('currency', 'EUR'),
         ('business coverage', 'N'),
         ('interest rate type', 'Z'),
         ('start', '?startPeriod=2012-01-01')]

param_joined = '.'.join(value for key, value in param[1:-1])

series_key = '{}/{}{}'.format(param[0][1],
                              param_joined,
                              param[-1][1])

## Request data

In [3]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get('{}{}'.format(url, series_key), headers=headers).json()

## Pandas DataFrame

In [4]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = [date['start'][:10] for date in date_list]
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][1]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '0:{}:0:0:0:0:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
df.index = dates
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2019-07-01,-0.1037,0.00,-0.39,0.354,-0.07,2.16,0.13,1.654,-0.206,0.443
2019-08-01,-0.3737,-0.28,-0.65,0.141,-0.34,1.98,-0.05,1.404,-0.499,0.173
2019-09-01,-0.3024,-0.24,-0.59,0.180,-0.28,1.50,-0.01,0.902,-0.427,0.204
2019-10-01,-0.1974,-0.16,-0.47,0.203,-0.16,1.34,0.02,1.004,-0.314,0.187
2019-11-01,-0.0861,-0.04,-0.35,0.386,-0.02,1.36,0.07,1.266,-0.192,0.349
